In [362]:
import pandas as pd
import numpy as np
fn = "a_solar.txt"
f = open(fn, "r")

In [363]:
#Line 1
l1 = f.readline()
print(l1)

5 3



In [364]:
W = int(l1.split()[0])
H = int(l1.split()[1])

In [365]:
map_conv = {'#' : 0, '_':1, 'M':2}
def read_map(f):
    map_int = []
    map_c = []
    for x in range(H):
        line = f.readline()
        int_line = [map_conv[i] for i in list(line)[:-1] ] 
        char_line = list(line)[:-1]
        map_int.append(int_line)
        map_c.append(char_line)
    return map_int, map_c

In [366]:
map_int, map_c = read_map(f)


In [367]:
D = int(f.readline().split()[0]) 

In [368]:
class Developer:
    def __init__(self,compagnia="",bonus=0,n_skills=0, skills=[] ):
        self.compagnia = compagnia
        self.bonus = bonus
        self.n_skills = n_skills
        self.skills = skills

In [369]:
developers = []
for n in range(D):
    l = f.readline()
    l_split = l.split()
    developer = Developer()
    developer.compagnia = l_split[0]
    developer.bonus = int(l_split[1])
    developer.n_skills = int(l_split[2])
    developer.skills = l_split[3:]
    developers.append(developer)
    

In [370]:
class ProjecManager:
    def __init__(self,compagnia="",bonus=0):
        self.compagnia = compagnia
        self.bonus = bonus
       

In [371]:
M = int(f.readline().split()[0]) 

In [372]:
pms = []
for n in range(M):
    l = f.readline()
    l_split = l.split()
    pm = ProjecManager()
    pm.compagnia = l_split[0]
    pm.bonus = int(l_split[1])
    pms.append(pm)
    

In [373]:
compagnia = [pm.compagnia for pm in pms]
bonus = [pm.bonus for pm in pms]
d = {'compagnia': compagnia, 'bonus': bonus}
pms_pandas = pd.DataFrame(d)

In [374]:
compagnia = [dev.compagnia for dev in developers]
bonus = [dev.bonus for dev in developers]
n_skills = [dev.n_skills for dev in developers]
skills = [dev.skills for dev in developers]
d = {'compagnia': compagnia, 'bonus': bonus, 'n_skills': n_skills, 'skills':skills}
dev_pandas = pd.DataFrame(d)

In [375]:
def points_dev_dev(developer1 = Developer(), developer2 = Developer()):
    common_skills = 0
    for i in range(developer1.n_skills):
        for j in range(i, developer2.n_skills):
            if developer1.skills[i] == developer2.skills[j]:
                common_skills = common_skills + 1
    diff_skills = developer1.n_skills + developer2.n_skills - (common_skills * 2)
    if developer1.compagnia == developer2.compagnia:
        bonus_potential = developer1.bonus * developer2.bonus
    else:
        bonus_potential = 0
    working_potential = common_skills * diff_skills
    points = bonus_potential + working_potential
    return points


def points_pm(developer1, developer2):
    if developer1.compagnia == developer2.compagnia:
        bonus_potential = developer1.bonus * developer2.bonus
    else:
        bonus_potential = 0
    return bonus_potential

def points_dev_dev_pandas(developer1, developer2):
    common_skills = 0
    for i in range(developer1['n_skills']):
        for j in range(i, developer2['n_skills']):
            if developer1['skills'][i] == developer2['skills'][j]:
                common_skills = common_skills + 1
    diff_skills = developer1['n_skills'] + developer2['n_skills'] - (common_skills * 2)
    if developer1['compagnia'] == developer2['compagnia']:
        bonus_potential = developer1['bonus'] * developer2['bonus']
    else:
        bonus_potential = 0
    working_potential = common_skills * diff_skills
    points = bonus_potential + working_potential
    return points


def points_pm_pandas(developer1, developer2):
    if developer1['compagnia'] == developer2['compagnia']:
        bonus_potential = developer1['bonus'] * developer2['bonus']
    else:
        bonus_potential = 0
    return bonus_potential


In [383]:
dev_pandas_sorted = dev_pandas.sort_values('compagnia', ascending=False)
pms_pandas_sorted = pms_pandas.sort_values('compagnia', ascending=False)

In [384]:
points_dev_dev_pandas(dev_pandas_sorted.iloc[0], dev_pandas_sorted.iloc[1])

58

In [385]:
sorted([('abc', 121),('abc', 231),('abc', 148), ('abc',221)], key=lambda x: x[1])


[('abc', 121), ('abc', 148), ('abc', 221), ('abc', 231)]

In [386]:
test = [('abc', 121),('abc', 231),('abc', 148), ('abc',221)]
match = [elem for elem in test if elem[1]==121]

In [501]:
def sortSecond(val):
    return val[2]


def find_couples():
    couple_points = []
    for i in range(0, 5):
        for j in range(i + 1, 5):
            points = points_dev_dev_pandas(dev_pandas_sorted.iloc[i], dev_pandas_sorted.iloc[j])
            couple_points.append([dev_pandas_sorted.index[i], dev_pandas_sorted.index[j], points, 0])
    couple_points.sort(key=sortSecond, reverse=True)
    return couple_points
couple_points = find_couples()



In [508]:
def find_n_best_matches(n, couple_points, dev_pandas_sorted):
    if len(couple_points)==0:
        return 0,[],couple_points
    idx = couple_points[0][0]
    matches = find_matches(idx,couple_points)
    n = n if len(matches) >= n else len(matches)
    if n == 0:
        return 0,[],couple_points
    for match in matches[:n]:
        #couple_points.pop(match[1])
        #print(match[1])
        dev_pandas_sorted.drop((match[0][0] != idx) * match[0][0] + (match[0][1] != idx) * match[0][1])
    output_id = [idx]+[(matches[n][0][0] != idx) * matches[n][0][0] + (matches[n][0][1] != idx) * matches[n][0][1] for n in range(n)]
    temp = []
    for j in couple_points:
        print(j)
        if (j[0]  in output_id) and (j[1]  in output_id):
            temp.append(j)
    couple_points = temp
    #for p,couple in enumerate(couple_points):
    #    if (couple[0] in output_id) or (couple[1] in output_id):
    #        print(p)
    #        couple_points.pop(p)
    return n+1, output_id, couple_points

def find_matches(elemid, array):
    ids =[]
    matches = []
    i = 0
    for elem in array:
        if elem[0]==elemid or elem[1]==elemid:
            matches.append((elem,i))
        i = i+1
        
    #matches = [(elem,n) for n,elem in enumerate(array) if elem[0]==elemid or elem[1]==elemid]
    #matches = sorted(matches, key=lambda x: x[2])
    return matches



In [509]:
 find_n_best_matches(2,couple_points,dev_pandas_sorted)

[0, 2, 58, 0]
[2, 7, 19, 0]
[0, 7, 14, 0]
[0, 3, 3, 0]
[4, 3, 3, 0]
[0, 4, 0, 0]
[2, 4, 0, 0]
[2, 3, 0, 0]
[7, 4, 0, 0]
[7, 3, 0, 0]


(3, [0, 2, 7], [[0, 2, 58, 0], [2, 7, 19, 0], [0, 7, 14, 0]])

In [492]:
0 in [0,2]

True

In [489]:
couple_points.pop(0)

[0, 3, 3, 0]

In [497]:
couple_points

[[0, 2, 58, 0],
 [2, 7, 19, 0],
 [0, 7, 14, 0],
 [0, 3, 3, 0],
 [4, 3, 3, 0],
 [0, 4, 0, 0],
 [2, 4, 0, 0],
 [2, 3, 0, 0],
 [7, 4, 0, 0],
 [7, 3, 0, 0]]

In [421]:
dev_pandas

,compagnia,bonus,n_skills,skills
0,opn,7,2,"[java, bpm]"
1,clstr,5,2,"[python, azure]"
2,opn,8,2,"[python, java]"
3,com_vl,4,3,"[java, cybersecurity, big_data]"
4,mac,1,2,"[nlp, big_data]"
5,clstr,3,2,"[azure, c#]"
6,com_vl,6,2,"[cybersecurity, python]"
7,opn,2,3,"[bpm, python, project_management]"
8,ble,5,4,"[java, c, sql, junit]"
9,clstr,1,4,"[python, c, java, bpm]"


In [280]:
couple_points[0]

[0, 4, 0, 0]